In [1]:
!pip install optuna -q
!pip install --upgrade datasets -q
!pip install --upgrade triton -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive') # mounting drive where files are stored

Mounted at /content/drive


In [3]:
import sys
path = "/content/drive/MyDrive/AI_Cybersecurity/Project/"
sys.path.append(path)

In [4]:
from sklearn.preprocessing import StandardScaler
import multiprocessing
import datasets
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict
import torch
from utils import BPE_Tokenizer

In [5]:
# Function to split the dataset into train, validation, and test sets
def train_val_test_split(dataset, train_val_ratio, val_test_ratio, stratify_column=None):
    # First split: Train/Validation
    train_val = dataset.train_test_split(test_size=train_val_ratio, stratify_by_column=stratify_column) if stratify_column is not None else dataset.train_test_split(test_size=train_val_ratio)
    # Second split: Validation/Test
    val_test = train_val['test'].train_test_split(test_size=val_test_ratio, stratify_by_column=stratify_column) if stratify_column is not None else train_val['test'].train_test_split(test_size=val_test_ratio)

    # Return a dictionary with train, validation, and test splits
    final_dataset = DatasetDict({
        'train': train_val['train'],
        'validation': val_test['train'],
        'test': val_test['test']
    })
    return final_dataset

# Adds a stratification label for better class balancing during splitting
def add_stratification_column(example):
    label = 'safe' if example['label'] == 1 else 'vulnerable'
    example['stratify_label'] = label # Copy target to 'stratify_label'
    return example

def add_metadata(example, columns):
    # Extract the values to normalize from the example
    metadata_values = [example[col] for col in columns]

    # Convert the normalized values to a tensor
    metadata_tensor = torch.tensor(metadata_values, dtype=torch.float32)

    # Add the metadata tensor to the example
    example['metadata'] = metadata_tensor
    return example

def process_dataset(example):
    """
    Combine 'url' and 'title' into a single formatted string for tokenization.
    """
    combined_string = f"<URL>{example['URL']} <TITLE>{example['Title']}"
    example['input_text'] = combined_string  # Add the combined string as a new column
    return example

def tokenize_dataset(example, max_length):
    """
    Tokenize and add `input_ids` and `attention_mask` as separate columns.
    """
    tokenized = tokenizer.encode(example['input_text'], max_length=max_length)
    example['input_ids'] = tokenized['input_ids']  # Assign input_ids to its own column
    example['attention_mask'] = tokenized['attention_mask']  # Assign attention_mask to its own column
    return example


In [6]:
urls = pd.read_csv("/content/drive/MyDrive/AI_Cybersecurity/Project/PhiUSIIL_Phishing_URL_Dataset.csv")
urls

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.000000,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.000000,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.000000,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.000000,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.000000,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235790,660997.txt,https://www.skincareliving.com,29,www.skincareliving.com,22,0,com,100.000000,1.000000,0.522907,...,1,0,1,51,7,21,187,2,191,1
235791,77185.txt,https://www.winchester.gov.uk,28,www.winchester.gov.uk,21,0,uk,100.000000,0.785714,0.028555,...,1,0,0,50,1,7,88,0,31,1
235792,622132.txt,https://www.nononsensedesign.be,30,www.nononsensedesign.be,23,0,be,100.000000,1.000000,0.003319,...,0,0,1,27,10,30,58,2,67,1
235793,7503962.txt,https://patient-cell-40f5.updatedlogmylogin.wo...,55,patient-cell-40f5.updatedlogmylogin.workers.dev,47,0,dev,28.157537,0.465116,0.000961,...,0,0,0,0,0,3,0,0,0,0


In [7]:
urls.columns

Index(['FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength', 'IsDomainIP',
       'TLD', 'URLSimilarityIndex', 'CharContinuationRate',
       'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain',
       'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio',
       'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL',
       'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
       'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'Title', 'DomainTitleMatchScore', 'URLTitleMatchScore',
       'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect',
       'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame',
       'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
       'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto',
       'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef',
       'NoOfEmptyRef', 'NoOf

In [8]:
text_feature_columns = urls.select_dtypes(include=['object', 'string']).columns.tolist()  # Text-based columns (URLs)
numerical_columns = urls.select_dtypes(include=['int64', 'float64']).columns.tolist()  # Numerical columns
target_column = 'label'  # The target column for classification
if target_column in text_feature_columns:
    text_feature_columns.remove(target_column)
if target_column in numerical_columns:
    numerical_columns.remove(target_column)

print("The text feature columns are:", text_feature_columns)
print("The numerical columns are:", numerical_columns)
print("The target column is:", target_column)

The text feature columns are: ['FILENAME', 'URL', 'Domain', 'TLD', 'Title']
The numerical columns are: ['URLLength', 'DomainLength', 'IsDomainIP', 'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']
The target column is: label


In [9]:
tokenizer = BPE_Tokenizer(directory=path+'Tokenizer')
vocab = tokenizer.vocab
vocab_size = len(vocab) + 1
print("Vocab size is", vocab_size)

Vocab size is 10007


In [10]:
print(f"There are {len(urls['label'])} total examples with {len(urls[urls['label'] == 0])} being classified as safe and {len(urls[urls['label'] == 0])} being classified as vulernable")

There are 235795 total examples with 100945 being classified as safe and 100945 being classified as vulernable


In [11]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data in the specified columns
urls[numerical_columns] = scaler.fit_transform(urls[numerical_columns])

urls

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,-0.086486,www.southbankmosaics.com,0.276436,-0.052087,com,0.744382,0.713153,1.043145,...,-0.557340,-0.155042,1.02681,0.099788,0.182551,0.783327,0.305222,-0.134778,0.464130,1
1,31372.txt,https://www.uni-mainz.de,-0.280125,www.uni-mainz.de,-0.597807,-0.052087,de,0.744382,-0.825555,-0.905197,...,-0.557340,-0.155042,1.02681,0.301270,0.035622,-0.113046,-0.147555,-0.134778,1.041672,1
2,597387.txt,https://www.voicefmradio.co.uk,-0.134896,www.voicefmradio.co.uk,0.057875,-0.052087,uk,0.744382,0.097670,-0.921472,...,-0.557340,-0.155042,1.02681,-0.202435,-0.057878,-0.157865,-0.130576,-0.021406,-0.274876,1
3,554095.txt,https://www.sfnmjournal.com,-0.207510,www.sfnmjournal.com,-0.269966,-0.052087,com,0.744382,0.713153,1.043145,...,1.794238,6.449862,1.02681,-0.290583,0.276051,0.200684,-0.243770,-0.078092,-0.113413,1
4,151578.txt,https://www.rewildingargentina.org,-0.038077,www.rewildingargentina.org,0.494997,-0.052087,org,0.744382,0.713153,-0.717171,...,1.794238,-0.155042,1.02681,2.744236,0.115765,1.052238,0.039216,-0.078092,0.221935,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235790,660997.txt,https://www.skincareliving.com,-0.134896,www.skincareliving.com,0.057875,-0.052087,com,0.744382,0.713153,1.043145,...,1.794238,-0.155042,1.02681,0.313862,0.008908,0.469596,0.690083,-0.021406,0.880209,1
235791,77185.txt,https://www.winchester.gov.uk,-0.159101,www.winchester.gov.uk,-0.051405,-0.052087,uk,0.744382,-0.276016,-0.921472,...,1.794238,-0.155042,-0.97389,0.301270,-0.071235,-0.157865,0.129771,-0.134778,-0.113413,1
235792,622132.txt,https://www.nononsensedesign.be,-0.110691,www.nononsensedesign.be,0.167156,-0.052087,be,0.744382,0.713153,-1.021764,...,-0.557340,-0.155042,1.02681,0.011639,0.048979,0.872964,-0.040021,-0.021406,0.110152,1
235793,7503962.txt,https://patient-cell-40f5.updatedlogmylogin.wo...,0.494430,patient-cell-40f5.updatedlogmylogin.workers.dev,2.789885,-0.052087,dev,-1.734996,-1.755936,-1.031133,...,-0.557340,-0.155042,-0.97389,-0.328361,-0.084592,-0.337140,-0.368284,-0.134778,-0.305927,0


In [12]:
urls_dataset = Dataset.from_pandas(urls)
urls_dataset.set_format(type='torch')

In [13]:
# Get the number of available CPU cores
num_cores = multiprocessing.cpu_count()

print(f"Number of available CPU cores: {num_cores}")

Number of available CPU cores: 8


In [14]:
prepared_dataset = urls_dataset.map(lambda x: add_metadata(x, numerical_columns), num_proc=num_cores)

Map (num_proc=8):   0%|          | 0/235795 [00:00<?, ? examples/s]

In [15]:
# Apply the function to create a new 'stratify_label' column
prepared_dataset = prepared_dataset.map(add_stratification_column, num_proc=num_cores)

Map (num_proc=8):   0%|          | 0/235795 [00:00<?, ? examples/s]

In [16]:
cleaned_dataset = prepared_dataset.remove_columns(numerical_columns)

In [17]:
# Convert the 'stratify_label' column to ClassLabel type (required for stratification)
unique_labels = sorted(cleaned_dataset.unique('stratify_label'))  # Get unique class labels
class_labels = ClassLabel(names=[str(label) for label in unique_labels])  # Create ClassLabel object for stratification

# Cast the 'stratify_label' column to ClassLabel type
cleaned_dataset = cleaned_dataset.cast_column("stratify_label", class_labels)

Casting the dataset:   0%|          | 0/235795 [00:00<?, ? examples/s]

In [18]:
# Split the dataset into train, validation, and test sets with stratification
split_dataset = train_val_test_split(cleaned_dataset, train_val_ratio=0.2, val_test_ratio=0.5, stratify_column='stratify_label')

In [19]:
# Apply the transformation to all splits in the DatasetDict
processed_dataset = split_dataset.map(process_dataset, num_proc=num_cores)

Map (num_proc=8):   0%|          | 0/188636 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/23579 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/23580 [00:00<?, ? examples/s]

In [ ]:
processed_dataset.save_to_disk(path+"Datasets/PreparedDataset/")

In [21]:
loaded_dataset = datasets.load_from_disk(path+"Datasets/PreparedDataset/")

In [22]:
max_length = 256

# Apply the function and unpack columns into the dataset
tokenized_dataset = loaded_dataset.map(lambda x: tokenize_dataset(x, max_length=max_length), num_proc=num_cores)

Map (num_proc=8):   0%|          | 0/188636 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/23579 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/23580 [00:00<?, ? examples/s]

In [23]:
output_example = 5
print(f"Original Example: {tokenized_dataset['train']['input_text'][output_example]}\nDecoded Example: {tokenizer.decode(tokenized_dataset['train']['input_ids'][output_example])}")

Original Example: <URL>https://www.pagepemblokiraaanannaanannanananaan.3gp.work.gd/ <TITLE>pagepemblokiraaanannaanannanananaan3gpwork
Decoded Example: https://www.pagepemblokiraaanannaanannanananaan.3gp.work.gd/ pagepemblokiraaanannaanannanananaan3gpwork


In [ ]:
tokenized_dataset.save_to_disk(path+"Datasets/FinalDataset/")